`pidgin.display` is a collection of display rules for IPython objects.  

`StringFormatter` is a class that adds reules for string `object`s.

In [1]:
    import abc, fnmatch, io, dataclasses, functools, html, IPython, jinja2, inspect, collections, inspect, importlib, types, base64, string, traitlets
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]
    with __import__('importnb').Notebook():
        try: 
            from .rendering import PidginDisplay
            from .templating import format_images
        except: 
            from rendering import PidginDisplay
            from templating import format_images
    ip = IPython.get_ipython()

In [2]:
def graphviz(ip, str): 
    try: import graphviz
    except: return
    if '->' in str and not str.startswith('di'): str = 'di' + str
    return graphviz.Source(str)

In [3]:
def eval_shorthand_ipython(ip, str):
    return eval(ip.input_transformer_manager.transform_cell(str), ip.user_ns, ip.user_global_ns)

In [4]:
def embed(ip, str):
    type = guess(str) or ''
    if str.startswith('http'): 
        if type.startswith('image') and not type.endswith('svg'):
            return IPython.display.Image(str)
        return IPython.display.IFrame(str, '100%', '500px')


In [5]:
class StringFormatterBase(abc.ABCMeta):
    level = 0
    def __enter__(cls): cls.level += 1
    def __exit__(cls, *exc): cls.level -= 1
        
    def format(cls, object, include=None, exclude=None, display=False):
        with cls:
            for type, callable in cls.formatters.items():
                if fnmatch.fnmatch(object, type):
                    result = callable(ip, object)
                    if result: 
                        data, metadata = ip.display_formatter.format(result)
                        return data, metadata
            if cls.level == 1: 
                return ip.display_formatter.format(PidginDisplay(object, ip=ip))

In [6]:
class StringFormatter(metaclass=StringFormatterBase):
    formatters = {'https://*' : embed,
                  'http://*' : embed,
                  'graph {*}': graphviz, 
                  'digraph {*}': graphviz,
                  ';[! ]*': eval_shorthand_ipython,
                  ',[! ]*': eval_shorthand_ipython,}

In [7]:
def format_module(module):
    return {
        'text/html': F"""<a title="{'&#xA;'.join(html.escape(module.__doc__).splitlines())}"><code>{html.escape(str(module))}</code></a>""", 'text/plain': str(module)}

In [8]:
def format_doc(object, **dict):
    if object.__doc__:
        if inspect.isfunction(object) or inspect.isclass(object): 
            name = object.__module__
        if isinstance(object, str):
            object = PidginDisplay(object.__doc__, ip=ip, ns=vars(importlib.import_module(name)))
        return ip.display_formatter.format(object.__doc__)
    return html.escape(str(object))

In [9]:
   def format_bytes(bytes, object):
        return {
            'text/html': format_images('image/png', {'image/png': bytes}),
            'text/plain': repr(object),}, {}

In [10]:
   def show_axes(object):
        ip = IPython.get_ipython()
        import matplotlib.backends.backend_agg
        bytes = io.BytesIO()
        matplotlib.backends.backend_agg.FigureCanvasAgg(
            getattr(object, 'figure', object)
        ).print_png(bytes)
        return format_bytes(bytes.getvalue(), object)

In [11]:
   def show_sympy_axes(object): 
        s = io.BytesIO()
        object.save(s)
        return format_bytes(s.getvalue(), object)

In [12]:
    def format_frame(DataFrame):
        return {
            'text/html': F"<span>{DataFrame._repr_html_().lstrip('<div>').rstrip('</div>')}</span>",
            'text/plain': repr(DataFrame)
        }, {}

In [13]:
    formatters_by_name = (
        ('matplotlib.figure', 'Axes', show_axes), 
        ('matplotlib.axes._subplots', 'AxesSubplot', show_axes),
        ('pandas.core.frame', 'DataFrame', format_frame),
        ('sympy.plotting.plot', 'Plot', show_sympy_axes),
    )

In [14]:
    def load_ipython_extension(ip): 
        mimebundle_formatter = ip.display_formatter.mimebundle_formatter
        mimebundle_formatter.for_type(type, format_doc)
        for object in formatters_by_name: mimebundle_formatter.for_type_by_name(*object)  
        mimebundle_formatter.for_type(types.ModuleType, format_module)
        mimebundle_formatter.for_type(str, StringFormatter.format)

    def unload_ipython_extension(ip=None): 
        for object in (type, types.ModuleType, str): 
            try: ip.display_formatter.mimebundle_formatter.type_printers.pop(object)
            except: ...
        for module, name, callable in formatters_by_name:
            try: ip.display_formatter.mimebundle_formatter.deferred_printers.pop((module, name))
            except: ...


    __name__ == '__main__' and load_ipython_extension(get_ipython())